In [ ]:
import pandas as pd
import numpy as np
import torch
import time
import matplotlib.pylab as plt
from tqdm import tqdm
import pickle

from shutil import copyfile
copyfile(src = "../input/jane-multiclass-models/Models.py", dst = "../working/Models.py")

from Models import *

In [ ]:
DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
#DEVICE = 'cpu'

In [ ]:
def fit(model, x):
    x = torch.tensor(x, dtype=torch.float)
    pred = model(x.to(DEVICE))
    return np.mean(pred.sigmoid().detach().cpu().numpy(), axis=1)

In [ ]:
target_cols = ['resp', 'resp_1', 'resp_2', 'resp_3', 'resp_4']
useless_cols = ['date', 'weight', 'ts_id']
target = 'action'
NFOLDS = 1
Model_Root = '../input/jane-multiclass-models/'

#train = pd.read_csv('../input/jane-street-market-prediction/train.csv', nrows=10000)
#feature_cols = list(train.drop(useless_cols + target_cols, axis=1))
with open(Model_Root + 'feature_cols.pkl', 'rb') as handle:
    feature_cols = pickle.load(handle)

f_mean = np.load(Model_Root + 'fmean.npy')

In [ ]:
models = []

model = torch.load(Model_Root + 'FOLD_1_10.pth')
models.append(model)
for ep in [50, 100, 150, 200]:
    model = torch.load(Model_Root + f'FOLD_1_10_{ep}.pth')
    models.append(model)


In [ ]:
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow as tf
import tensorflow_addons as tfa


def create_mlp(
    num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate
):

    inp = tf.keras.layers.Input(shape=(num_columns,))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dropout(dropout_rates[0])(x)
    for i in range(len(hidden_units)):
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
        x = tf.keras.layers.Dropout(dropout_rates[i + 1])(x)
    
    x = tf.keras.layers.Dense(num_labels)(x)
    out = tf.keras.layers.Activation("sigmoid")(x)

    model = tf.keras.models.Model(inputs=inp, outputs=out)
    model.compile(
        optimizer=tfa.optimizers.RectifiedAdam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing),
        metrics=tf.keras.metrics.AUC(name="AUC"),
    )

    return model

In [ ]:
hidden_units = [160, 160, 160]
dropout_rates = [0.2, 0.2, 0.2, 0.2]
label_smoothing = 1e-2
learning_rate = 1e-3

SEED=101
tf.keras.backend.clear_session()
tf.random.set_seed(SEED)
clf = create_mlp(
    len(feature_cols), 5, hidden_units, dropout_rates, label_smoothing, learning_rate
    )

clf = load_model('../input/keras-overfitted-model/model.h5')

In [ ]:
import janestreet
env = janestreet.make_env()


for (test_df, pred_df) in tqdm(env.iter_test()):

    if test_df['weight'].item() > 0:
        
        x_tst = test_df.loc[:, feature_cols].values
        
        if np.isnan(x_tst[:, :].sum()):
            x_tst[:, 1:] = np.nan_to_num(x_tst[:, 1:]) + np.isnan(x_tst[:, 1:]) * f_mean
        
        pred = np.zeros((1, len(target_cols)))
        for model in models:
            outs = model(torch.tensor(x_tst, dtype=torch.float).to(DEVICE))
            label = []
            for i in range(len(pred[0])):
                label.append(torch.max(outs[0][i], 1)[1].unsqueeze(1))
            label = torch.cat(label, dim=1)
            pred += label.detach().cpu().numpy()/len(models)
        
        pred = np.median(pred, axis=1)
        pred_df.action = np.where(pred > 2, 1, 0).astype(int)

    else:

        pred_df.action = 0

    env.predict(pred_df)
